In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
import pickle
import pandas as pd
import numpy as np

<h1>Load hasil baca berita</h1>

In [3]:
def loadPickle(name):
    file = open(name, 'rb')
    data = pickle.load(file)
    file.close()
    
    return data

In [4]:
def listOfDictToDataFrame(listOfDict):
    dataFrame = pd.DataFrame(listOfDict)
    
    x = dataFrame.iloc[:, :-1].values
    y = dataFrame.iloc[:, 4].values
    
    return dataFrame, x, y

<h1>Bangun Label encoder</h1>

In [5]:
forX = preprocessing.LabelEncoder()
forY = preprocessing.LabelEncoder()
forNewX = preprocessing.LabelEncoder()

In [6]:
def labelEncoderX(x):
    temp = x[0] #[ 0 'sss' 'sss' 'sss']
    x = np.delete(x,0) #['sss' 'sss' 'sss']
    
    forX.fit(x)
    x = forX.transform(x) #[2 2 1]
    x = np.append(x,temp) #[2 2 1 0]
    
    return x

In [7]:
def labelEncoderNewX(x):
    temp = x[0] #[ 0 'sss' 'sss' 'sss']
    x = np.delete(x,0) #['sss' 'sss' 'sss']
    
    forNewX.fit(x)
    x = forNewX.transform(x) #[2 2 1]
    x = np.append(x,temp) #[2 2 1 0]
    
    return x

<h1>Baca Data Test</h1>

In [8]:
def extFeature(words):
    listOfDictFeature = []
    i = 0
    for i in range(len(words)): #iterasi dari awal tiap kata
        dictFeature = {}
        
        dictFeature = inputToDict(words, i)
        
        listOfDictFeature.append(dictFeature)
        
    return listOfDictFeature

def inputToDict(words, i):
    dictFeature = {}
    
    dictFeature['index'] = i    
    dictFeature['first_upper_case'] = isFirstUpper(words[i])
    dictFeature['prev_word'] = isEdge(i, words)
    dictFeature['word'] = words[i]
    
    return dictFeature

def isFirstUpper(word):
    if word[0].isupper():
        return 'y'
    else:
        return 'n'
    
def isEdge(i, words):
    if i == 0:
        return 'null'
    else:
        return words[i-1]

In [9]:
def readPerWord(file):
    words = []
    f = open(file, 'r') #open file
    for line in f:
        words = line.split(' ') #tiap satu baris split per kata
    
    f.close()
    
    words[len(words)-1] = words[len(words)-1].replace('\n',' ')

    return words

<h1>Proses ==============</h1>

In [10]:
data = loadPickle("training.pickle")
dataFrame, x, y = listOfDictToDataFrame(data)

In [11]:
#forX biar string jadi skala
#tiap 1 list dijadikan angka dengan label encoder
for i in range (len(x)):
    temp = []
    temp = labelEncoderX(x[i])
    x[i] = temp

In [12]:
#forY biar string jadi skala
forY = preprocessing.LabelEncoder()
forY.fit(y)
y = forY.transform(y)

<h3>Bangun classifier KNN</h3>

In [13]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [14]:
testWord = readPerWord("test.txt")
listOfDictFeature = extFeature(testWord)
df = pd.DataFrame(listOfDictFeature)

#mengubah data frame ke list
newdf = df.iloc[:, :].values

In [15]:
#forNewX biar string jadi skala
#tiap 1 list dijadikan angka dengan label encoder
for i in range (len(newdf)):
    temp = []
    temp = labelEncoderNewX(newdf[i])
    newdf[i] = temp

<h3>Prediksi</h3>

In [16]:
y_pred = classifier.predict(newdf)

In [17]:
print(y_pred)

[3 3 3 3 3 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1]


In [18]:
y_pred_invers = list(forY.inverse_transform(y_pred))
print(y_pred_invers)

['person', 'person', 'person', 'person', 'person', 'o', 'o', 'o', 'o', 'o', 'o', 'location', 'location', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'organization', 'organization', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']


<h3>Hasil</h3>

In [19]:
person = []
location = []
organization = []
for i in range(len(df)):
    if y_pred_invers[i] == 'person':
        person.append(df['word'][i])
    elif y_pred_invers[i] == 'location':
        location.append(df['word'][i])
    elif y_pred_invers[i] == 'organization':
        organization.append(df['word'][i])

In [20]:
print(person)
print(location)
print(organization)

['Dari', 'Arab', 'Saudi,', 'Habib', 'Rizieq']
['Subianto', 'mengaku']
['kata', 'Prabowo']
